# A Simple NN to classify Images of Faces

In [1]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def load_set(path):
    arr = []
    for filename in glob.glob(path):
        im=Image.open(filename)
        arr.append(im.copy())
        im.close()    
    return(arr)

valid_set = load_set('faces/good/*.png')
ngtv_set = load_set('faces/bad/*.png')

#### Since there are fewer images in the valid set, I've created augmented mirroed duplicates

In [3]:
rarr = [img.transpose(Image.FLIP_LEFT_RIGHT) for img in valid_set]
valid_set.extend(rarr)

#### I've created a balanced set of valid and negative images, and a list of labels with corresponding lables

In [4]:
minlen = min(len(valid_set), len(ngtv_set))

vset = [np.array(image) for image in valid_set[:minlen]]
nset = [np.array(image) for image in ngtv_set[:minlen]]
allset = np.array(vset + nset)
labels = [1]*len(vset)+[0]*len(nset)
x = np.expand_dims(allset, 3)
y = np.expand_dims(labels, 1)
y = to_categorical(y)

#### The data is divided into 80% train, 20% test. All the sets' data normilized

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.2)

datagen = image.ImageDataGenerator(rescale=1.0/255.0)
train_iterator = datagen.flow(x_train, y_train, batch_size=32)
test_iterator = datagen.flow(x_test, y_test, batch_size=32)

#### Adding model layers sequentialy
This is a convolutional neural network. 

The architacture is VGGNET, with filters of sizes 32 --> 64 --> 128 --> 2. 

I used maxpooling to down-sample the features and droupout to drop neurons so as to avoid over-fitting.


In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(50,50, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### The loss is fitted for a two-class problem. This optimizer had the best results

In [7]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001) ,metrics=['accuracy'])

In [8]:
history = model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=10,
                              validation_data=(test_iterator))

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 23s 230ms/step - loss: 0.3914 - acc: 0.8121 - val_loss: 0.1919 - val_acc: 0.9280
Epoch 2/10
100/100 [==============================] - 21s 208ms/step - loss: 0.1622 - acc: 0.9405 - val_loss: 0.1824 - val_acc: 0.9431
Epoch 3/10
100/100 [==============================] - 23s 225ms/step - loss: 0.1068 - acc: 0.9663 - val_loss: 0.0811 - val_acc: 0.9731
Epoch 4/10
100/100 [==============================] - 22s 220ms/step - loss: 0.0892 - acc: 0.9721 - val_loss: 0.0588 - val_acc: 0.9844
Epoch 5/10
100/100 [==============================] - 23s 226ms/step - loss: 0.0721 - acc: 0.9782 - val_loss: 0.0646 - val_acc: 0.9743
Epoch 6/10
100/100 [==============================] - 23s 227ms/step - loss: 0.0529 - acc: 0.9812 - val_loss: 0.0876 - val_acc: 0.9737
Epoch 7/10
100/100 [==============================] - 23s 229ms/step - loss: 0.0604 - acc: 0.9815 - val_loss: 0.0420 - val_acc: 0.9875
Epoch 8

In [9]:
model.save_weights('model_wieghts.h5')
model.save('model_keras.h5')

#### I created a final test set to test prediction. It chooses randomly from the valid and negative sets

In [10]:
import random
testset = np.array(random.sample(vset,10) + random.sample(nset,10))
tx = np.expand_dims(testset, 3)

In [11]:
model.predict_classes(tx)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [12]:
import datetime as dt
n1=dt.datetime.now()
model.predict(tx)
n2=dt.datetime.now()
print("average time for image prediction in millisecond: ", (n2-n1).microseconds/(len(tx)*1000))

average time for image prediction in millisecond:  2.40015
